In [5]:
import os
import sys

import torch
import transformers

/home/thecoolooser/anaconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import foundation model form huggigface 

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilgpt2-finetuned-finance")
model = AutoModelForCausalLM.from_pretrained("lxyuan/distilgpt2-finetuned-finance")

# Read pdf file using pyPDFLoader

In [10]:
from langchain_community.document_loaders import PyPDFLoader

In [11]:
# read pdf file
loader  = PyPDFLoader("../data/raw/Finance_Bill.pdf")

In [14]:
# split data into pages
pages = loader.load_and_split()

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [50]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
    separators=['\n\n', '\n', '(?=>\.)', ' ','']
)
docs = text_splitter.split_documents(pages)

In [57]:
for doc in docs:
    print(doc.page_content)

BILL No. 55 OF 2024 
 
 
 
 
 
 
 
 
 
 
 
THE FINANCE (NO. 2) BILL, 2024 
 
 
 
 
 
 
 
 
 
 
 
(AS INTRODUCED IN LOK SABHA)
THE FINANCE (No. 2) BILL, 2024 
_______ 
 
ARRANGEMENT OF CLAUSES 
______ 
 
CHAPTER I 
 
PRELIMINARY 
CLAUSES 
 
1. Short title and commencement. 
 
CHAPTER II 
 
RATES OF INCOME-TAX 
2. Income-tax. 
 
CHAPTER III 
 
DIRECT TAXES 
 
Income-tax 
3. Amendment of section 2. 
4. Amendment of section 10. 
5. Amendment of section 11. 
6. Amendment of section 12A. 
7. Amendment of section 12AB. 
8. Amendment of section 12AC. 
9. Amendment of section 13. 
10. Amendment of section 16. 
11. Amendment of section 28. 
12. Amendment of section 36. 
13. Amendment of section 37. 
14. Amendment of section 40. 
15. Amendment of section 43D. 
16. Amendment of section 44B. 
17. Insertion of new section 44BBC. 
18. Amendment of section 46A. 
19. Amendment of section 47. 
20. Amendment of section 48. 
21. Amendment of section 50AA. 
22. Amendment of section 55. 
23. Amendment of se

In [66]:
# extraact page content
docs_text = [doc.page_content for doc in docs]

In [67]:
type(docs_text)

list

In [71]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings to accommodate the new token

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [72]:
# Tokenize text
inputs = tokenizer(docs_text, return_tensors="pt", truncation=True, max_length=1024, padding=True)

In [ ]:
# Generate embeddings
with torch.no_grad():
    outputs = model(**inputs)
    
# Use the [CLS] token embedding as the sentence embedding
embeddings = outputs.last_hidden_state[:, 0, :]

In [34]:
from langchain_huggingface import HuggingFaceEmbeddings

In [36]:
hf = HuggingFaceEmbeddings(
    model_name='lxyuan/distilgpt2-finetuned-finance',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
    )

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [28]:
embeddings = SentenceTransformerEmbeddings(model_name="lxyuan/distilgpt2-finetuned-finance")

/tmp/ipykernel_14659/4091128036.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="lxyuan/distilgpt2-finetuned-finance")


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [20]:
token_ids = tokenizer.encode(docs)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]